<a href="https://colab.research.google.com/github/pavass/Optimized_CNN_Model/blob/master/OptimizeCNNModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install keras.tuner

     |████████████████████████████████| 61kB 2.0MB/s 
  Created wheel for keras.tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=a015f1c2910ad115eb6aeb000aaf5e4544e86ac8a3dafe56452718f4dbe43111
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=d40f1b3e43210b6dbafb3e6bcf6a5df91971f55d9dec350e0ec283cab7902569
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras.tuner terminaltables


In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [0]:
print(tf.__version__)

2.2.0-rc4


In [0]:
fashion_mnist=keras.datasets.fashion_mnist

In [0]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [0]:
train_images=train_images/255.0
test_images=test_images/255.0

In [0]:
train_images[0].shape

(28, 28)

In [0]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [0]:
def build_model(hp):
  model=keras.Sequential([
      keras.layers.Conv2D(
          filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
          kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
          activation='relu',
          input_shape=(28,28,1)
      ),
      keras.layers.Conv2D(
          filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
          kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
          activation='relu'
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
          activation='relu'
      ),
      keras.layers.Dense(10,activation='softmax')
  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model
  


In [0]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [0]:
tuner_search=RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name="Mnist Fashion")

In [0]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.4125 - accuracy: 0.8502 - val_loss: 0.3211 - val_accuracy: 0.8810
Epoch 2/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2682 - accuracy: 0.9013 - val_loss: 0.2579 - val_accuracy: 0.9018
Epoch 3/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2150 - accuracy: 0.9198 - val_loss: 0.2531 - val_accuracy: 0.9080


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3819 - accuracy: 0.8629 - val_loss: 0.2846 - val_accuracy: 0.8933
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2445 - accuracy: 0.9100 - val_loss: 0.2611 - val_accuracy: 0.9048
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1869 - accuracy: 0.9308 - val_loss: 0.2401 - val_accuracy: 0.9132


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.4065 - accuracy: 0.8531 - val_loss: 0.3196 - val_accuracy: 0.8855
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2711 - accuracy: 0.9002 - val_loss: 0.2717 - val_accuracy: 0.8943
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2202 - accuracy: 0.9186 - val_loss: 0.2531 - val_accuracy: 0.9067


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.5421 - accuracy: 0.8020 - val_loss: 0.4392 - val_accuracy: 0.8302
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.4201 - accuracy: 0.8436 - val_loss: 0.4311 - val_accuracy: 0.8403
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3927 - accuracy: 0.8538 - val_loss: 0.4042 - val_accuracy: 0.8533


Epoch 1/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.4303 - accuracy: 0.8454 - val_loss: 0.3798 - val_accuracy: 0.8648
Epoch 2/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.3322 - accuracy: 0.8768 - val_loss: 0.3463 - val_accuracy: 0.8793
Epoch 3/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.3014 - accuracy: 0.8890 - val_loss: 0.3631 - val_accuracy: 0.8775


INFO:tensorflow:Oracle triggered exit


In [0]:
model=tuner_search.get_best_models(num_models=1)[0]

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       1120      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        64576     
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                1769520   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                490       
Total params: 1,835,706
Trainable params: 1,835,706
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1360 - accuracy: 0.9492 - val_loss: 0.2695 - val_accuracy: 0.9077
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0966 - accuracy: 0.9642 - val_loss: 0.2998 - val_accuracy: 0.9213
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0705 - accuracy: 0.9737 - val_loss: 0.3371 - val_accuracy: 0.9157
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0523 - accuracy: 0.9810 - val_loss: 0.3807 - val_accuracy: 0.9115
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0384 - accuracy: 0.9857 - val_loss: 0.4213 - val_accuracy: 0.9087
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0302 - accuracy: 0.9895 - val_loss: 0.4657 - val_accuracy: 0.9120
Epoch 10/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0253 - accuracy: 0.9912 - val_loss: 0.5356 - val_accuracy